## Tool Calling with Agentic AI - AutoGen

### LLM Used - Granite3.0-8B

In this notebook we will learn how to use Tool Calling with Agentic AI in order to solve different problems.

Tool-calling agents expand the capabilities of an LLM by allowing it to interact with external systems. This approach empowers agents to dynamically solve problems by utilizing tools, accessing memory, and planning multi-step actions.

Tool calling agents enable:

1. Multi-Step Decision Making: The LLM can orchestrate a sequence of decisions to achieve complex objectives.
2. Tool Access: The LLM can select and use various tools as needed to interact with external systems and APIs.

This architecture allows for more dynamic and flexible behaviors, enabling agents to solve complex tasks by leveraging external resources efficiently.

In [1]:
!pip -q install autogen-agentchat~=0.2


[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import autogen
from typing import Literal

from pydantic import BaseModel, Field
from typing_extensions import Annotated

import autogen
from autogen.cache import Cache

INFERENCE_SERVER_URL = os.getenv('API_URL_GRANITE')
MODEL_NAME = "granite30-8b"
API_KEY= os.getenv('API_KEY')

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In [3]:
# Configuration for the vLLM endpoint
local_llm_config = {
    "config_list": [
        {
            "model": MODEL_NAME,
            "api_key": "EMPTY", 
            "base_url": f"{INFERENCE_SERVER_URL}/v1"
        }
    ],
    "cache_seed": None,
    "temperature": 0.01,
    "timeout": 600,
}

In [4]:
from autogen import ConversableAgent

agent = ConversableAgent(
    "chatbot",
    llm_config=local_llm_config,
    code_execution_config=False,  # Turn off code execution, by default it is off.
    function_map=None,  # No registered functions, by default it is None.
    human_input_mode="NEVER",  # Never ask for human input.
)

reply = agent.generate_reply(messages=[{"content": "What is Agentic AI.", "role": "user"}])
print(reply)

[autogen.oai.client: 12-05 12:46:04] {351} WARNING - Model granite30-8b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
{'content': 'Agentic AI refers to artificial intelligence systems that are designed to act autonomously and make decisions on their own, rather than being controlled by humans. These systems are often equipped with advanced learning capabilities and can adapt to new situations and environments. They are used in various fields such as robotics, autonomous vehicles, and healthcare.', 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': []}


In [5]:
from autogen import UserProxyAgent, ConversableAgent


chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="For currency exchange tasks, only use the functions you have been provided with. Reply TERMINATE when the task is done.",
    llm_config=local_llm_config,
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },
)

CurrencySymbol = Literal["USD", "EUR"]

def exchange_rate(base_currency: CurrencySymbol, quote_currency: CurrencySymbol) -> float:
    if base_currency == quote_currency:
        return 1.0
    elif base_currency == "USD" and quote_currency == "EUR":
        return 1 / 1.1
    elif base_currency == "EUR" and quote_currency == "USD":
        return 1.1
    else:
        raise ValueError(f"Unknown currencies {base_currency}, {quote_currency}")


In [6]:
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },
)


from pydantic import BaseModel, Field
from typing_extensions import Annotated

class Currency(BaseModel):
    currency: Annotated[str, Field(..., description="Currency symbol")]
    amount: Annotated[float, Field(..., description="Amount of currency", ge=0)]


# another way to register a function is to use register_function instead of register_for_execution and register_for_llm decorators
def currency_calculator(
    base: Annotated[Currency, "Base currency: amount and currency symbol"],
    quote_currency: Annotated[CurrencySymbol, "Quote currency symbol"] = "USD",
) -> Currency:
    quote_amount = exchange_rate(base.currency, quote_currency) * base.amount
    return Currency(amount=quote_amount, currency=quote_currency)


autogen.agentchat.register_function(
    currency_calculator,
    caller=chatbot,
    executor=user_proxy,
    description="Currency exchange calculator.",
)

In [7]:
chatbot.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'Currency exchange calculator.',
   'name': 'currency_calculator',
   'parameters': {'type': 'object',
    'properties': {'base': {'$ref': '#/definitions/Currency',
      'definitions': {'Currency': {'title': 'Currency',
        'type': 'object',
        'properties': {'currency': {'title': 'Currency',
          'description': 'Currency symbol',
          'type': 'string'},
         'amount': {'title': 'Amount',
          'description': 'Amount of currency',
          'minimum': 0,
          'type': 'number'}},
        'required': ['currency', 'amount']}},
      'description': 'Base currency: amount and currency symbol'},
     'quote_currency': {'enum': ['USD', 'EUR'],
      'type': 'string',
      'default': 'USD',
      'description': 'Quote currency symbol'}},
    'required': ['base']}}}]

In [8]:
with Cache.disk() as cache:
    # start the conversation
    res = user_proxy.initiate_chat(
        chatbot, message="How much is 112.23 Euros in US Dollars?", summary_method="reflection_with_llm", cache=cache
    )

user_proxy (to chatbot):

How much is 112.23 Euros in US Dollars?

--------------------------------------------------------------------------------
[autogen.oai.client: 12-05 12:46:17] {351} WARNING - Model granite30-8b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
chatbot (to user_proxy):

To calculate the exchange rate from Euros to US Dollars, I will use the "currency_calculator" function.

Here's the JSON input for the function:

```json
{
  "base": {
    "currency": "EUR",
    "amount": 112.23
  },
  "quote_currency": "USD"
}
```

Now, I will call the function and provide the result.

```python
import json
import requests

def currency_calculator(input_data):
    url = "https://api.exchangerate-api.com/v4/latest/USD"
    response = requests.get(url)
    data = response.json()
    base_currency = input_data["base"]["currency"]
    amount = input_data["base"]["amount"]
    rat

In [9]:
print("Chat summary:", res.summary)

Chat summary: {'content': 'The user asked for the conversion of 112.23 Euros to US Dollars, and the assistant provided the result as 112.23 USD.', 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': []}
